In [22]:
import requests
import pandas as pd
import sqlite3

In [23]:
server = "https://map.sepa.org.uk/server/rest/services/Open/Environmental_Monitoring/MapServer/1/query?f=json&where=(1%3D1)%20AND%20(1%3D1)&returnGeometry=true&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&outSR=4326&resultOffset=0&resultRecordCount=100"


In [24]:
data = requests.get(server, headers={"Accept": "application/json, text/javascript, */*; q=0.01"})

In [25]:
df = pd.json_normalize(data.json()['features'])
df = df.rename(columns={
    "attributes.OBJECTID": "id",
    "attributes.DESCRIPTION": "name",
    "geometry.x": "long",
    "geometry.y": "lat"
})\
.set_index("id")\
.drop(columns=[
    'attributes.CLASS_DESCRIPTION', 'attributes.BW_URL',
    'attributes.YEAR'
])
df['alternate_name'] = df['name']
locations = df[[
    'name', 'alternate_name', 'lat', 'long'
]].copy()
locations

,name,alternate_name,lat,long
id,,,,
1,Coldingham,Coldingham,55.892068,-2.133192
2,Crail (Roome Bay),Crail (Roome Bay),56.261350,-2.616211
3,Cullen Bay,Cullen Bay,57.694543,-2.835047
4,Culzean,Culzean,55.348838,-4.808215
5,Dhoon Bay,Dhoon Bay,54.814230,-4.090692
...,...,...,...,...
86,Lunan Bay,Lunan Bay,56.655171,-2.502350
904,Barassie,Barassie,55.556522,-4.658773
905,Lower Largo,Lower Largo,56.211867,-2.934186


In [26]:
locations

,name,alternate_name,lat,long
id,,,,
1,Coldingham,Coldingham,55.892068,-2.133192
2,Crail (Roome Bay),Crail (Roome Bay),56.261350,-2.616211
3,Cullen Bay,Cullen Bay,57.694543,-2.835047
4,Culzean,Culzean,55.348838,-4.808215
5,Dhoon Bay,Dhoon Bay,54.814230,-4.090692
...,...,...,...,...
86,Lunan Bay,Lunan Bay,56.655171,-2.502350
904,Barassie,Barassie,55.556522,-4.658773
905,Lower Largo,Lower Largo,56.211867,-2.934186


In [27]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


89